In [2]:
#!pip install tenseal

In [50]:
import tenseal as ts

In [127]:
#the context is required for doing computation on encrypted data , it contains a store for the keys and parameters..
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes =[20,40,40,20])

In [78]:
secret_key = context.secret_key()

In [79]:
import numpy as np 

In [80]:
plain_tensor = np.random.randn(2,3)
plain_tensor

array([[-1.11332515, -1.01092288, -2.657617  ],
       [ 0.03515662,  1.54415344,  1.95509673]])

In [81]:
encrypted_tensor = ts.ckks_tensor(context,plain_tensor,scale=2**40)
encrypted_tensor

In [82]:
# we can set the scale directly to the context that we are not obliged to set it every time we encrypt a tensor 
context.global_scale = 2 ** 40
encrypted_tensor = ts.ckks_tensor(context , plain_tensor)
encrypted_tensor

In [83]:
print(encrypted_tensor.decrypt())

The PlainTensor object is a tensor that we mainly use internally to represent tensors with plain values. You can always call `tolist()` to convert it to a list.

In [84]:
print(encrypted_tensor.decrypt().tolist())

[[-1.1133251533742394, -1.0109228787129558, -2.657616998795217], [0.03515661890384239, 1.544153441374982, 1.9550967274385511]]


In [85]:
print(encrypted_tensor.decrypt().size())

2


In [86]:
print(plain_tensor)

[[-1.11332515 -1.01092288 -2.657617  ]
 [ 0.03515662  1.54415344  1.95509673]]


 # Computation and Evaluation

In [87]:
encrypted_result = (encrypted_tensor + 2) *3 - plain_tensor

In [88]:
expected_result = (plain_tensor +2)*3 - plain_tensor

In [89]:
expected_result

array([[3.77334969, 3.97815424, 0.684766  ],
       [6.07031324, 9.08830688, 9.91019346]])

In [90]:
np.array(encrypted_result.decrypt().tolist())

array([[3.77335005, 3.97815464, 0.68476574],
       [6.07031406, 9.08830831, 9.91019505]])

In [91]:
vector1 = np.random.randn(3)
vector2 = np.random.randn(3)
enc_vec1 = ts.ckks_tensor(context,vector1)
enc_vec2 = ts.ckks_tensor(context,vector2)
print(f"result : {enc_vec1.dot(enc_vec2).decrypt().tolist()}")
print(f"expected result : {vector1.dot(vector2)}")
print(vector1)
print(vector2)

result : 1.2288798970043284
expected result : 1.2288797331227481
[-1.21012702 -0.70528408  1.26909879]
[ 0.42186361 -1.99115657  0.26401196]


In [92]:
matrix1 = np.random.randn(3,3)
matrix2 = np.random.randn(3,3)
enc_matrix1 = ts.ckks_tensor(context,matrix1)
enc_matrix2 = ts.ckks_tensor(context,matrix2)
print(f"result : \n\t{(enc_matrix1 * enc_matrix2).decrypt().tolist()}")
print(f"\nexpected result : \n\t{matrix1 * matrix2}")

result : 
	[[-0.6208705093502983, -0.1376900552200658, 0.6935937608120599], [-0.6257555395308259, 2.357346199218757, 0.1321815627583215], [-0.015341571103966571, -0.41518303423984415, -0.5125634960670463]]

expected result : 
	[[-0.62087043 -0.13769004  0.69359367]
 [-0.62575546  2.35734588  0.13218155]
 [-0.01534157 -0.41518298 -0.51256343]]


## Batch Computation

In [126]:
from time import time 


1633831219.6752121

In [94]:
# a single ciphertext can hold up to `poly_modulus_degree / 2` values
# so let's use all the slots available
batch_size= 8192 //2
mat1 = np.random.randn(batch_size , 3,3)
mat2 =np.random.randn(3,3)
# batch is by default set to False, we have to turn it on to use the packing feature of ciphertexts
enc_mat1 = ts.ckks_tensor(context,mat1,batch=True)
enc_mat2 = ts.ckks_tensor(context,mat2)
print(f"result : {enc_mat1.dot(enc_mat2).decrypt().tolist()[0]}")
print(f"expected result : {mat1.dot(mat2)[0]}")

#when we set batch=False , it take more time and memory to compute the results then when we enable it 




result : [[-0.12790044514024634, 0.048676312523775245, 0.95731815299766], [-2.463882144925088, 1.7820485133607953, 3.627484445861768], [3.2163420902238244, -2.382104423153012, -2.3448437677602936]]
expected result : [[-0.12790043  0.04867631  0.95731802]
 [-2.46388181  1.78204828  3.62748396]
 [ 3.21634166 -2.3821041  -2.34484346]]


In [22]:
# TenSEAL use the parelle computation of the computer systeme by default but we can modiying it when creating the context
non_parallel_context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60],
    n_threads=1,
)

In [23]:
# for the Decryption, the context make all the keys private , we can turn it to public but we must save the key for the decryption
sk = context.secret_key()
context.make_context_public()
# by making the context public we dropped the secret key from it, so for the next decryption we need to pass it 

In [24]:
enc_mat1.decrypt()

ValueError: the current context of the tensor doesn't hold a secret_key, please provide one as argument

the Decryption does not work and that because the context does not have a secret key

In [27]:
enc_mat1.decrypt(sk)


You should always make a context public before sending it to other parties to compute on encrypted data.

# Serialization

if we want to send encrypted data or the context we can use the serialize method, every sendable object can be serializable via the serrialize method  

In [28]:
ser_context = context.serialize()
type(ser_context)

bytes

In [34]:
ser_tensor = encrypted_tensor.serialize()
type(ser_tensor)

bytes

there is also a method for deszeialization the context 

In [39]:
loadded_context = ts.context_from(ser_context)
loadded_context 

the tensors must be linked to a context to work properly and this is using the method below

In [42]:
loadded_tensors = ts.ckks_tensor_from(loadded_context,ser_tensor)
loadded_tensors

However, there is also a way to do it the lazy way, deserializing, then linking it to a specific context

In [45]:
lazy_loadded_tensor = ts.lazy_ckks_tensor_from(ser_tensor)
lazy_loadded_tensor + 5

ValueError: missing context

we see that we are not able to manipulate the tensor because it's not linked to a context, so linking the tensor to a context is an obligation for working with the tensors

In [46]:
lazy_loadded_tensor.link_context(loadded_context)


In [47]:
lazy_loadded_tensor + 5

In [167]:
65544

65544

# Playing with the parameters

We are going now to set juste 1 thread to work and disable the paralelle computation, and see if it will take more time then when working with multiple thread 

In [168]:
non_parallel_context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=16384,
    coeff_mod_bit_sizes=[60, 40, 40, 60],
    n_threads=1,
)
non_parallel_context.global_scale = 2**40

In [162]:
start = time()
matrix1 = np.random.randn(100000,3,3)
matrix2 = np.random.randn(3,3)
enc_matrix1 = ts.ckks_tensor(non_parallel_context,matrix1,batch=True)
enc_matrix2 = ts.ckks_tensor(non_parallel_context,matrix2)
product_result = enc_matrix1.dot(enc_matrix2).decrypt().tolist()
product_expected = matrix1.dot(matrix2)
end = time()

print(f"Time consumed : {end - start}")



ValueError: can't encrypt vectors of this size, please use a larger polynomial modulus degree.

We see that when we are using a vector of 100000 the upper bound is surpassed so we need to increase the poly_modulus_degree parameter

In [220]:
non_parallel_context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=16384,
    coeff_mod_bit_sizes=[60, 40, 40, 60],
#     n_threads=1,
    n_threads=4,

)
non_parallel_context.global_scale = 2**40 # 2**80

In [221]:
start = time()
matrix1 = np.random.randn(1000,10,3)
matrix2 = np.random.randn(3,3)
enc_matrix1 = ts.ckks_tensor(non_parallel_context,matrix1,batch=True)
enc_matrix2 = ts.ckks_tensor(non_parallel_context,matrix2)
product_result = enc_matrix1.dot(enc_matrix2).decrypt().tolist()
product_expected = matrix1.dot(matrix2)
end = time()

print(f"Time consumed : {end - start}")



ValueError: scale out of bounds

We can see that when we increase the numbers of thread we increase the time of execution  

When setting the scale to 2\*\*80 we have an error that says "scale out of bounds" so this scale is very large for our data , using 2**40 is quite perfect